In [ ]:
# generate watersheds from the points (this takes a few moments)

sql = """
CREATE TABLE hydrostn_wsd AS
SELECT
  stn.id,
  wsd1.wscode_ltree,
  wsd1.localcode_ltree,
  ST_union(wsd2.geom) as geom
FROM public.hydrostn stn
INNER JOIN whse_basemapping.fwa_watersheds_poly_sp wsd1
ON ST_Intersects(stn.geom, wsd1.geom)
INNER JOIN whse_basemapping.fwa_watersheds_poly_sp wsd2
ON
  -- b is a child of a, always
  wsd2.wscode_ltree <@ wsd1.wscode_ltree
AND
    -- conditional upstream join logic, based on whether watershed codes are equivalent
  CASE
    -- first, consider simple case - streams where wscode and localcode are equivalent
     WHEN
        wsd1.wscode_ltree = wsd1.localcode_ltree
     THEN TRUE
     -- next, the more complicated case - where wscode and localcode are not equal
     WHEN
        wsd1.wscode_ltree != wsd1.localcode_ltree AND
        (
         -- tributaries: b wscode > a localcode and b wscode is not a child of a localcode
            (wsd2.wscode_ltree > wsd1.localcode_ltree AND
             NOT wsd2.wscode_ltree <@ wsd1.localcode_ltree)
            OR
         -- capture side channels: b is the same watershed code, with larger localcode
            (wsd2.wscode_ltree = wsd1.wscode_ltree
             AND wsd2.localcode_ltree >= wsd1.localcode_ltree)
        )
      THEN TRUE
  END
WHERE wsd1.watershed_group_code = 'SALM'
GROUP BY
  stn.id,
  wsd1.wscode_ltree,
  wsd1.localcode_ltree"""
db.execute(sql)

In [ ]:
# display the watersheds and stations on the map
wsdgdf = gpd.read_postgis('SELECT id, ST_Transform(geom, 4326) as geom FROM hydrostn_wsd', db.engine)
f, ax = plt.subplots(1)
wsdgdf.plot(axes=ax)
#stngdf.plot(axes=ax)
mplleaflet.display(tiles='esri_worldtopo') 